## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

## Prepare data for model training

In [2]:
with open("../input/tps-may-data-preprocess-v5/TPS_May_Dataset.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

del processed_data
gc.collect()

0

In [3]:
cat_cols = ['feature_0','feature_1','feature_2','feature_3','feature_4','feature_5','feature_6','feature_7',
            'feature_8','feature_9','feature_10','feature_11','feature_12','feature_13','feature_14','feature_15',
            'feature_16','feature_17','feature_18','feature_19','feature_20','feature_21','feature_22','feature_23',
            'feature_24','feature_25','feature_26','feature_27','feature_28','feature_29','feature_30','feature_31',
            'feature_32','feature_33','feature_34','feature_35','feature_36','feature_37','feature_38','feature_39',
            'feature_40','feature_41','feature_42','feature_43','feature_44','feature_45','feature_46','feature_47',
            'feature_48','feature_49','clusters__0','clusters__1','clusters__2','clusters__3','clusters__4',
            'clusters__5','clusters__6','clusters__7','clusters__8','clusters__9','clusters__10','clusters__11',
            'clusters__12','clusters__13','clusters__14']

train_df[cat_cols] = train_df[cat_cols].astype(int)
test_df[cat_cols] = test_df[cat_cols].astype(int)
cat_cols_indices = [train_df.columns.get_loc(col) for col in cat_cols]
print(cat_cols_indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [4]:
Xtrain = train_df.loc[:, train_df.columns != 'target'].copy()
Ytrain = train_df['target'].copy()
Ytrain_oh = pd.get_dummies(train_df['target']).copy()
Xtest = test_df.copy()

print("Xtrain: {} \nYtrain: {} \nYtrain_oh: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, 
                                                                   Ytrain_oh.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (99918, 100) 
Ytrain: (99918,) 
Ytrain_oh: (99918, 4) 
Xtest: (50000, 100)


20

## Build and validate the model

In [5]:
FOLD = 10
NUM_SEED = 3

# Prediction Clipping Thresholds
p_min = 0.025
p_max = 1 - p_min

np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_cb = np.zeros((Ytrain.shape[0], 4))
y_pred_final_cb = np.zeros((Xtest.shape[0], 4))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain.values, Ytrain.values)):
        counter += 1

        train_x, train_y, train_y_oh = Xtrain.iloc[train], Ytrain.iloc[train], Ytrain_oh.iloc[train]
        val_x, val_y, val_y_oh = Xtrain.iloc[val], Ytrain.iloc[val], Ytrain_oh.iloc[val]

        model = CatBoostClassifier(
            objective='MultiClass',
            eval_metric='MultiClass',
            num_boost_round=5000,
            max_ctr_complexity=15,
            od_wait=3000, 
            od_type='Iter',
            use_best_model=True,
            bootstrap_type='Poisson',
            learning_rate=0.01465,
            reg_lambda=0.01864,
            subsample=0.68135,
            max_depth=6, 
            min_data_in_leaf=1,
            task_type='GPU',
            verbose=0
        )

        model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
                  cat_features=cat_cols_indices,
                  early_stopping_rounds=200, verbose=200)

        y_pred = model.predict_proba(val_x)
        y_pred = np.clip(y_pred, p_min, p_max)
        y_pred_meta_cb[val] += y_pred
        y_pred_final_cb += model.predict_proba(Xtest)
        
        score = log_loss(val_y_oh, y_pred)
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_cb = y_pred_meta_cb / float(NUM_SEED)
y_pred_final_cb = y_pred_final_cb / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

0:	learn: 1.3775507	test: 1.3775693	best: 1.3775693 (0)	total: 24.4ms	remaining: 2m 2s
200:	learn: 1.1003286	test: 1.1050249	best: 1.1050249 (200)	total: 3.42s	remaining: 1m 21s
400:	learn: 1.0880942	test: 1.0980668	best: 1.0980668 (400)	total: 7.32s	remaining: 1m 23s
600:	learn: 1.0798820	test: 1.0952478	best: 1.0952478 (600)	total: 11.2s	remaining: 1m 22s
800:	learn: 1.0733082	test: 1.0938982	best: 1.0938982 (800)	total: 15.2s	remaining: 1m 19s
1000:	learn: 1.0670908	test: 1.0930703	best: 1.0930703 (1000)	total: 18.9s	remaining: 1m 15s
1200:	learn: 1.0613917	test: 1.0926429	best: 1.0926294 (1193)	total: 22.3s	remaining: 1m 10s
1400:	learn: 1.0560600	test: 1.0924679	best: 1.0924679 (1400)	total: 26s	remaining: 1m 6s
1600:	learn: 1.0510792	test: 1.0922113	best: 1.0922038 (1599)	total: 29.7s	remaining: 1m 3s
1800:	learn: 1.0459610	test: 1.0921206	best: 1.0920901 (1766)	total: 33.1s	remaining: 58.7s
2000:	learn: 1.0412647	test: 1.0920140	best: 1.0919738 (1962)	total: 36.4s	remaining: 54.

In [6]:
np.savez_compressed('./CB_Meta_Features.npz',
                    y_pred_meta_cb=y_pred_meta_cb, 
                    oof_score=oof_score,
                    y_pred_final_cb=y_pred_final_cb)

## Create submission file

In [7]:
y_pred_final_cb = np.clip(y_pred_final_cb, p_min, p_max)

test_df = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
submit_df = pd.DataFrame()
submit_df['id'] = test_df['id']
submit_df['Class_1'] = y_pred_final_cb[:,0]
submit_df['Class_2'] = y_pred_final_cb[:,1]
submit_df['Class_3'] = y_pred_final_cb[:,2]
submit_df['Class_4'] = y_pred_final_cb[:,3]
submit_df.head()

,id,Class_1,Class_2,Class_3,Class_4
0,100000,0.092508,0.621406,0.164237,0.121849
1,100001,0.090627,0.667442,0.149546,0.092384
2,100002,0.083678,0.627592,0.187996,0.100733
3,100003,0.088087,0.531836,0.280991,0.099086
4,100004,0.074706,0.625882,0.179786,0.119626


In [8]:
submit_df.to_csv("./CB_submission.csv", index=False)